# Data Analyis of Canva Data
Here, we will be clearning, transforming and modelling the canva data.

In [7]:
import pandas as pd
import matplotlib as mpl
import numpy as np
from IPython.display import display, HTML
import pycountry

mau_df = pd.read_csv("mau_by_plan_type.csv")
device_tiers_df = pd.read_csv("device_tiers.csv")
canva_templates_df = pd.read_csv("canva_templates.csv")
population_df = pd.read_csv("global_population.csv")
mau_df.head()
population_df.head()


,Country Name,2024
0,Aruba,107624.0
1,Africa Eastern and Southern,769294618.0
2,Afghanistan,42647492.0
3,Africa Western and Central,521764076.0
4,Angola,37885849.0


## Stage 1: Cleaning
### 1. MAUs: Duplicate Average, Removing Unkowns
We have noticed several duplicate rows in the MAU dataset with differing MAU numbers. Since we cannot assume which entry is correct, and it also doesnt seem to be the result of seperate observations that require aggregation, we will simply average entries with duplicate Month + Plan Type + Country Code.

In [8]:
def code_to_name(code):
    try:
        return pycountry.countries.lookup(code).name
    except:
        return None

# group similar rows together
mau_df = (
    mau_df
      .groupby(['MONTH_END_DATE', 'PRIMARY_PLAN_TYPE', 'COUNTRY_CODE'], as_index=False)
      ['Monthly Active Users']
      .mean()
)

# typecast as int
mau_df['Monthly Active Users'] = np.ceil(mau_df['Monthly Active Users']).astype(int)

# convert code to country name
mau_df['COUNTRY_CODE'] = mau_df['COUNTRY_CODE'].apply(code_to_name)

# rename COUNTRY_CODE to COUNTRY for consistency between tables (for joining later)
mau_df.rename(columns={'COUNTRY_CODE': 'COUNTRY', 'Monthly Active Users': 'MAUs'}, inplace=True)

# cleaning global population data
population_df.rename(columns={'Country Name': 'COUNTRY', '2024': 'TOTAL_POPULATION'}, inplace=True)
population_df["TOTAL_POPULATION"] = population_df["TOTAL_POPULATION"].astype('Int64')

# reset indexes
mau_df = mau_df[mau_df["PRIMARY_PLAN_TYPE"] != "Unknown"].reset_index(drop=True).sort_values(by = ['COUNTRY', 'MONTH_END_DATE', 'PRIMARY_PLAN_TYPE'])
mau_df.head(10)

,MONTH_END_DATE,PRIMARY_PLAN_TYPE,COUNTRY,MAUs
2,2020-01-31,Canva Pro,Afghanistan,5
89,2020-01-31,Canva Pro - NFP,Afghanistan,2
245,2020-01-31,Education,Afghanistan,3
428,2020-02-29,Canva Pro,Afghanistan,6
529,2020-02-29,Canva Pro - NFP,Afghanistan,1
615,2020-02-29,Canva for Teams,Afghanistan,1
692,2020-02-29,Education,Afghanistan,5
995,2020-03-31,Canva Pro - NFP,Afghanistan,1
1150,2020-03-31,Education,Afghanistan,4
2060,2020-05-31,Domain,Afghanistan,1


### 2. Device Tiers: Matching Table Headers
We notice that in the Device Tiers table, countries are referred to by name rather than code. To preserve consistency, we will convert names to code.

In [ ]:
def name_to_code(name):
    try:
        return pycountry.countries.lookup(name).alpha_2
    except:
        return None


device_tiers_df = device_tiers_df.reset_index(drop=True).sort_values(by="Country")
device_tiers_df[['High', 'Mid', 'Low', 'Unknown']] = device_tiers_df[['High', 'Mid', 'Low', 'Unknown']].astype('Int64')
device_tiers_df.head()

# renaming headings to match other dataframes
device_tiers_df.rename(columns={'Country': 'COUNTRY', 'High': 'HIGH_DEVICE_TIER', 'Mid': 'MID_DEVICE_TIER', 
                                'Low': 'LOW_DEVICE_TIER', 'Unknown': 'UNKNOWN_DEVICE_TIER', 'Total': 'TOTAL'}, inplace=True)

# cleaning global population data
population_df.rename(columns={'Country Name': 'COUNTRY', '2024': 'TOTAL_POPULATION'}, inplace=True)
population_df["TOTAL_POPULATION"] = population_df["TOTAL_POPULATION"].astype('Int64')

# joining global population data to device tiers data
device_tiers_df = device_tiers_df.merge(population_df[['COUNTRY', 'TOTAL_POPULATION']], on='COUNTRY', how='left')
device_tiers_df.head()


,COUNTRY,HIGH_DEVICE_TIER,MID_DEVICE_TIER,LOW_DEVICE_TIER,UNKNOWN_DEVICE_TIER,Total,TOTAL_POPULATION
0,Afghanistan,6491,6277,1948,554,15270,42647492
1,Albania,23814,3914,432,1958,30118,2714617
2,Algeria,85198,76059,29307,6684,197248,46814308
3,American Samoa,407,115,9,94,625,46765
4,Andorra,3565,694,33,588,4880,81938
